In [0]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose( #Here, we are chaining different image transformatio together using Compose
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [0]:
import torch.nn as nn
import torch.nn.functional as F


#New neural net with an additional layer. CONV1 --> CONV2 --> POOL1 --> CONV3 --> POOL3 --> FC1 --> FC2 --> FC3

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 96, 5) #First convolutional layer with 3 inpout channels (RGB), 96 output and 5 filter size
        self.conv2 = nn.Conv2d(96, 32, 5) #Second convolutional layer with 96 input channels, 32 output and 5 filter size
        self.conv3 = nn.Conv2d(32, 16, 5) #Third convolutional layer with  32 input channels, 16 output and 5 filter size

        self.pool = nn.MaxPool2d(2, 2) # Max Pooling with 2 size and 2 strides, to be used after conv2 and again after conv3.

        self.fc1 = nn.Linear(16 * 4 * 4, 120) # 4*4 are height and width of each of the 16 output channels
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) #note that the output features for the final linear layer was choosen as 10 since we are dealing with 10 classes

        
    def forward(self, x):

        x = F.relu(self.conv1(x)) # first the convolutional operations is applied for the input signal and is followed by a ReLu activation. No pooling.

        x = self.pool(F.relu(self.conv2(x))) # second convolutional layer with max pooling
  
        x = self.pool(F.relu(self.conv3(x))) # third convolutional layer with max pooling

  
        x = x.view(-1, 16 * 4 * 4) #here we are re-shaping since we are coming from a conflate as input to a linear layer
        #the tensor arrives to the first linear layer with a reduced size of 4x4 due to the convolution and the pooling operations.

        # fully connected linear layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x


net = Net() #creating an instance of our network class

In [0]:
import torch.optim as optim 

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.001)

!pip install livelossplot --quiet
from livelossplot import PlotLosses
dataloaders = {
    "train": trainloader,
    "testing": testloader
}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def getnumcorrect (preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()


num_epochs=30       # run over 30 epochs
min_loss = 1        # set our minimum loss for stopping 
waiting_loss = 0    # used to calculate stopping
n_epochs_stop = 5   # if there is no improvement on the loss for 5 straight epochs we end the process
epochs_no_improve = 0  # counts how many epochs in a row it goes without loss improvment
early_stop= False     # set to false to start
correctones=[]        # initialize an array for correctones
losses=[]             # initialize an array for losses
liveloss = PlotLosses()
net = net.to(device)
for epoch in range(num_epochs):
        logs = {}
        for phase in ['train', 'testing']:
            if phase == 'train':
                net.train()
            else:
                net.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = net(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            

                if waiting_loss < running_loss < min_loss:
                  epochs_no_improve += 1

                if epochs_no_improve == n_epochs_stop:
                   print('Early stopping!' )
                   early_stop = True
                   break
                else:
                   continue

                waiting_loss = running_loss
            if early_stop:
              print("Stopped")
              break  

            adjust_learning_rate(epoch)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)
            
            prefix = ''
            if phase == 'testing':
                prefix = 'val_'

            logs[prefix + 'loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        
        liveloss.update(logs)
        liveloss.send()


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 4222, in parseImpl
    ret = e._parse(instring, loc, doActions)
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 1669, in _parseNoCache
    loc, tokens = self.parseImpl(instring, preloc, doActions)
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 4037, in parseImpl
    loc, exprtokens = e._parse(instring, loc, doActions)
  File "/usr/local/lib/python3.6/dis

KeyboardInterrupt: ignored

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %


In [0]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 66 %
Accuracy of   car : 76 %
Accuracy of  bird : 49 %
Accuracy of   cat : 47 %
Accuracy of  deer : 63 %
Accuracy of   dog : 43 %
Accuracy of  frog : 71 %
Accuracy of horse : 61 %
Accuracy of  ship : 77 %
Accuracy of truck : 74 %
